In [494]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown

In [495]:
class OneToOneITU():
    def __init__(self, n, m, size_params, random_seed, lbs=(1, 1), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 

        # generate problem
        np.random.seed(random_seed)

        self.A_ij = np.random.randint(1, size_params[0], size=[n, m])
        self.B_ij = np.random.randint(0, size_params[1], size=[n, m])

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij
    
    def d_ij(self, U_i, V_j, i, j):
        return U_i[i] + self.A_ij[i, j] * V_j[j] - self.B_ij[i, j]

    def U_ij(self,i_idx , V_j):
        return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]

    def V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]



#### Checking modules

The following functions check equilibrium conditions for a candidate $(\mu, U, V).$. Below I will refer to the equilibrium conditions as follows:
* (*Primal Feasibility*) $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ (and $\mu_{ij} \in \{0,1\}$),
* (*Primal Complementary Slackness*) $D_{ij}(U_i,V_j) = 0$ if $ \mu_{ij}  = 1$, 
* (*Dual Feasibility*) $D_{ij}(U_i,V_j) \geq 0$,
* (*Dual Complementary Slackness*) $U_i =  U_0 $ if $\sum_j \mu_{ij} = 0$ and $V_i =  V_0 $ if $\sum_i \mu_{ij} = 0,$ 
* (*Individual Rationality*) $U_i \geq  U_0 ,$ $V_i \geq  V_0 .$ 

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [496]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.mean((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.mean((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\frac{1}{{n}} \sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\frac{1}{{m}}\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [497]:
def check_dual_feas(self,U_i,V_j, output = None):

    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.mean()}'))
    if output:
        return D_feas, np.where(D_feas<0 )
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [498]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.mean()}'))
    if output:
        return P_CS_ij

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [499]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [500]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [501]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

# Parallel auction

In [518]:
n_small = 3
m_small = 4
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (10,10) , random_seed= 88988, lbs =(2,1), tol = .01)

In [634]:

V_0 = np.zeros(m_small)
mu_0 = np.zeros([n_small,m_small+1], dtype= bool)
unassigned = mu_0.sum(1) == 0


In [661]:

U_ij_s = np.concatenate((example_small.U_ij(unassigned,V_0), np.ones((unassigned.sum(),1))), axis = 1)

sorted_w_id = np.argsort(U_ij_s, axis = 1)[:,-2:]
sorted_w_val = np.take_along_axis(U_ij_s,sorted_w_id, axis = 1)
j_star = sorted_w_id[:,1]

In [662]:
U_ij_s

array([], shape=(0, 5), dtype=float64)

In [663]:
sorted_w_id

array([], shape=(0, 2), dtype=int64)

In [664]:
offers = example_small.V_ij(unassigned, j_star, sorted_w_val[:,0]) 
np.maximum.at(V_0, j_star, offers + example_small.tol)
V_0

array([3.01, 0.51, 0.81, 0.  ])

In [650]:
j_star

array([0])

In [651]:
matched_unassigned = (offers[:,None] + example_small.tol == V_0)
mu_0[unassigned,:-1] = matched_unassigned



In [652]:
perm_unmatched = (j_star == n_small+1)
mu_0[unassigned,-1] = perm_unmatched


In [654]:
unassigned = mu_0.sum(1) == 0
unassigned

array([False, False, False])

In [645]:
mu_0

array([[False, False,  True, False, False],
       [False,  True, False, False, False],
       [False, False, False, False, False]])

In [680]:
def ITU_auction(self):
    V_j = np.zeros(self.m)
    mu_ij = np.zeros([self.n,self.m+1])

    unassigned = mu_ij.sum(1) == 0
    iter = 0
    while unassigned.sum() > 0:
        
   
        U_ij_s = np.concatenate((self.U_ij(unassigned,V_j), np.ones((unassigned.sum(),1)) * self.lb_U), axis = 1)
        sorted_w_id = np.argsort(U_ij_s, axis = 1)[:,-2:]
        sorted_w_val = np.take_along_axis(U_ij_s,sorted_w_id, axis = 1)
        j_star = sorted_w_id[:,1]
    
        offers = self.V_ij(unassigned, j_star, sorted_w_val[:,0]) 
        np.maximum.at(V_j, j_star, offers + self.tol)


        matched_unassigned = (offers[:,None] + example_small.tol == V_j)
        mu_ij[unassigned,:-1] = matched_unassigned

        perm_unmatched = (j_star == n_small+1)
        mu_ij[unassigned,-1] = perm_unmatched

        unassigned = mu_ij.sum(1) == 0
        iter += 1

    matched_i_s = mu_ij[:,-1] == 0


    U_i = self.U_ij(matched_i_s, V_j)

    return mu_ij[:,:-1], U_i, V_j



OneToOneITU.ITU_auction = ITU_auction


In [681]:
example_small.ITU_auction()

(array([[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[-2.308e+01, -1.570e+00,  2.950e+00,  3.000e+00],
        [-1.000e-02,  7.980e+00,  4.760e+00,  3.000e+00],
        [ 3.990e+00,  1.960e+00,  3.140e+00,  4.000e+00]]),
 array([3.01, 0.51, 0.81, 0.  ]))